# 1. Collecting data

## 1.1 Importing useful libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
import time
pd.set_option('display.max_columns', None)

## 1.2 Custom functions to collect data

In [2]:
def extract_feature(css_path,numerical,soup):
    # Auxiliar function used in *get_sales_data()* to extract the extra features from a car
    if numerical == True:
        feature = soup.select(css_path)[0]
        feature = re.findall(r'\d+[.]?[,]?\d*',feature.get_text().strip())[0].replace('.','').replace(',','.')
    else:
        feature = soup.select(css_path)[0].get_text().strip()
    
    return feature

In [3]:
def get_sales_data(location,car_class,n_pages,sleeptime = 2.0):
    # We create the empty database
    data = pd.DataFrame(columns=['year','cv','km','fuel','doors','gearbox','emissions',\
                                 'color','warranty','seller','id','brand','price','boot',\
                                 'length','height','width','seats','max_sp','cmixto',\
                                'curban','extraurban','0-100','autonomy','cilindrada','cilindros',\
                                'transmission','max_par','marchas','class','location','link'])
 
    # Iterating over each page n_pages pages
    ncar = 0
    
    # CSS Path for several features
    boot_path = "div.index-card__technical-data:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2)"
    length_path = "div.index-card__technical-data:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)"
    height_path = "div.index-card__technical-data:nth-child(2) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)"
    width_path = "div.index-card__technical-data:nth-child(2) > div:nth-child(2) > div:nth-child(4) > div:nth-child(2)"
    seats_path = "div.index-card__technical-data:nth-child(2) > div:nth-child(2) > div:nth-child(6) > div:nth-child(2)"
    maxspeed_path = "div.index-card__technical-data:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2)"
    cmix_path = "#trackingIndexCardConsumption"
    curban_path = "div.index-card__technical-data:nth-child(3) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)"
    extraurban_path = "div.index-card__technical-data:nth-child(3) > div:nth-child(2) > div:nth-child(4) > div:nth-child(2)"
    zero100_path = "div.index-card__technical-data:nth-child(3) > div:nth-child(2) > div:nth-child(5) > div:nth-child(2)"
    aut_path = "div.index-card__technical-data:nth-child(3) > div:nth-child(2) > div:nth-child(6) > div:nth-child(2)"
    cilindrada_path = "div.index-card__technical-data:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)"
    cils_path = "div.index-card__technical-data:nth-child(4) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)"
    trans_path = "div.index-card__technical-data:nth-child(4) > div:nth-child(2) > div:nth-child(4) > div:nth-child(2)"
    maxpar_path = "div.index-card__technical-data:nth-child(4) > div:nth-child(2) > div:nth-child(5) > div:nth-child(2)"
    marcha_path = "div.index-card__technical-data:nth-child(4) > div:nth-child(2) > div:nth-child(6) > div:nth-child(2)"
    #ALL
    paths =[boot_path,length_path, height_path,width_path,seats_path,maxspeed_path,cmix_path,curban_path,\
            extraurban_path,zero100_path,aut_path,cilindrada_path, cils_path,trans_path,maxpar_path,marcha_path]
    # Booleans
    paths_bool = [True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True]
    
    
    if car_class == 'common':
        class_string = 'pequeno-mediano-grande-familiar-monovolumen'
    elif car_class == '4x4':
        class_string = 'coches-4x4-todoterreno'
    elif car_class == 'commercial':
        class_string = 'furgonetas-segunda-mano'
    elif car_class == 'sport':
        class_string = 'coches-deportivos-segunda-mano'
    else:
        print('ERROR. The provided class does not exist. Please choose "common","4x4","commercial" or "sport"')
        return
    
    baselink = "https://www.coches.com/coches-segunda-mano/"+class_string+"-en-"+location+".htm?page="
    print("Starting from: \n"+baselink+"0")
    for i in range(n_pages):
        link = baselink+str(i)      
        
        # Finding all the car from this page:
        while True:
            try:
                response = requests.get(link)
                break
            except:
                print('Retrying to connect...')
                sleep(abs(np.random.normal(sleeptime+2,0.15)))
                continue
        
        soup = BeautifulSoup(response.content, "html.parser")
        sleep(abs(np.random.normal(sleeptime,0.15)))
        car_links = soup.select("html body main.content-page div#vo-results.vo-results--rebranding div.pillList.vo-results__card-list.script__vo-results-card-list div.cc-car-card.vo-results__card.pill.script__pill")

        # Iterating for each car
        for car_link in car_links:
            if car_link.a['href'] == '':
                break # If this card has no car associated, move on
            
            # Extract ID
            car_id = re.findall('id=([\d]+)', car_link.a['href'])[0]
            if car_id in data['id'].values:
                break # if we've already scanned this sale, move on 
            
            # read the html of the car post
            while True:
                try:
                    response = requests.get(car_link.a['href'])
                    break
                except:
                    print('Retrying to connect...')
                    sleep(abs(np.random.normal(sleeptime+2,0.15)))
                    continue
            
            soup = BeautifulSoup(response.content, "html.parser")
            sleep(abs(np.random.normal(sleeptime,0.15)))
            
            # GENERAL FEATURES
            features_html = soup.select(".cc-car-overview")
            if len(features_html)==0:
                break # If we can't find the features, move on 
            features = features_html[0].select("p")
            car_features = [code.get_text() for i,code in enumerate(features) if (i+1)%2==0]

            # BRAND
            car_brand = soup.select("h1.index-card__make-model")[0].get_text()
            car_brand = re.findall(r"[\w]+",car_brand)[0]

            # PRICE
            car_price = [a.get_text() for a in soup.select("div.index-card__price-number")]
            car_prices = [int(re.findall(r'\d+[.]*\d*',price)[0].replace(".","")) for price in car_price if len(re.findall(r'\d+[.]*\d*',price))>0]

            # EXTRA FEATURES TABLE               
            some_features = [extract_feature(path,var_bool,soup=soup) for path, var_bool in zip(paths,paths_bool)]
            more_features = [car_id, car_brand, np.max(car_prices)]+some_features+[car_class, location, car_link.a['href']]
            
            car_features = car_features + more_features

            data.loc[ncar] = car_features
            ncar += 1


        # Show progress
        print(round((i+1)/n_pages*100,1),'% complete.',end='\r')
    print(f"Obtained {data.shape[0]} items of class '{car_class}'.")   
    return data

# Madrid

In [ ]:
madrid_vans = get_sales_data(location='madrid',car_class='commercial',n_pages=57,sleeptime=0)

In [ ]:
madrid_sports = get_sales_data(location='madrid',car_class='sport',n_pages=81,sleeptime=0)

In [ ]:
madrid_4x4 = get_sales_data(location='madrid',car_class='4x4',n_pages=500,sleeptime=0)

In [ ]:
%%time
madrid_common = get_sales_data(location='madrid',car_class='common',n_pages=500,sleeptime=0)

In [ ]:
madrid = pd.concat([madrid_common,madrid_4x4,madrid_sports,madrid_vans], axis=0)

In [ ]:
#madrid.to_csv('./data/madrid.csv',index=False)

# Barcelona

In [ ]:
%%time
barcelona_vans = get_sales_data(location='barcelona',car_class='commercial',n_pages=61,sleeptime=0)

In [ ]:
%%time
barcelona_sports = get_sales_data(location='barcelona',car_class='sport',n_pages=50,sleeptime=0)

In [ ]:
%%time
barcelona_4x4 = get_sales_data(location='barcelona',car_class='4x4',n_pages=500,sleeptime=0)

In [ ]:
%%time
barcelona_common = get_sales_data(location='barcelona',car_class='common',n_pages=500,sleeptime=0)

In [ ]:
barcelona = pd.concat([barcelona_common,barcelona_4x4,barcelona_sports,barcelona_vans], axis=0).reset_index(drop=True)

In [ ]:
#barcelona.to_csv('./data/barcelona.csv',index=False)

# Valencia

In [ ]:
%%time
valencia_vans = get_sales_data(location='valencia',car_class='commercial',n_pages=30,sleeptime=0)

In [ ]:
%%time
valencia_sports = get_sales_data(location='valencia',car_class='sport',n_pages=42,sleeptime=0)

In [ ]:
%%time
valencia_4x4 = get_sales_data(location='valencia',car_class='4x4',n_pages=369,sleeptime=0)

In [ ]:
%%time
valencia_common = get_sales_data(location='valencia',car_class='common',n_pages=500,sleeptime=0)

In [ ]:
valencia = pd.concat([valencia_common,valencia_4x4,valencia_sports,valencia_vans], axis=0).reset_index(drop=True)



In [ ]:
valencia.to_csv('./data/valencia.csv',index=False)


In [ ]:
valencia

# Sevilla

In [4]:
sevilla_vans = get_sales_data(location='sevilla',car_class='commercial',n_pages=41,sleeptime=0)

Starting from: 
https://www.coches.com/coches-segunda-mano/furgonetas-segunda-mano-en-sevilla.htm?page=0
Obtained 786 items of class 'commercial'.


In [5]:
sevilla_sports = get_sales_data(location='sevilla',car_class='sport',n_pages=50,sleeptime=0)

Starting from: 
https://www.coches.com/coches-segunda-mano/coches-deportivos-segunda-mano-en-sevilla.htm?page=0
Obtained 968 items of class 'sport'.


In [6]:
sevilla_4x4 = get_sales_data(location='sevilla',car_class='4x4',n_pages=393,sleeptime=0)

Starting from: 
https://www.coches.com/coches-segunda-mano/coches-4x4-todoterreno-en-sevilla.htm?page=0
Obtained 7066 items of class '4x4'.


In [7]:
sevilla_common = get_sales_data(location='sevilla',car_class='common',n_pages=500,sleeptime=0)

Starting from: 
https://www.coches.com/coches-segunda-mano/pequeno-mediano-grande-familiar-monovolumen-en-sevilla.htm?page=0
Obtained 8050 items of class 'common'.


In [8]:
sevilla = pd.concat([sevilla_common,sevilla_4x4,sevilla_sports,sevilla_vans], axis=0).reset_index(drop=True)


In [9]:
sevilla.to_csv('./data/sevilla.csv',index=False)

In [10]:
sevilla

,year,cv,km,fuel,doors,gearbox,emissions,color,warranty,seller,id,brand,price,boot,length,height,width,seats,max_sp,cmixto,curban,extraurban,0-100,autonomy,cilindrada,cilindros,transmission,max_par,marchas,class,location,link
0,04/2017,130 CV,51.417 km,Gasolina,5 puertas,manual,108 gr/m3,Plateado,SÍ,Profesional,7579469,Peugeot,14490,420,425,147,180,5,210,5.0,6.3,4.2,10.2,0,1199,3 en línea,Manual,230,6,common,sevilla,https://www.coches.com/coches-segunda-mano/oca...
1,2015,115 CV,147.365 km,Diesel,5 puertas,manual,117 gr/m3,negro,SÍ,Profesional,7574203,Ford,11200,363,436,148,182,5,0,4.5,5.7,3.7,10.8,0,1560,4 en línea,Manual,270,6,common,sevilla,https://www.coches.com/coches-segunda-mano/oca...
2,2017,258 CV,92.405 km,Diesel,4 puertas,automática secuencial,159 gr/m3,gris,SÍ,Profesional,7560591,Mercedes,37900,540,492,148,185,5,250,6.0,7.6,5.1,6.0,0,2987,6 en V,Automática secuencial,620,9,common,sevilla,https://www.coches.com/coches-segunda-mano/oca...
3,11/2012,110 CV,89.811 km,Gasolina,5 puertas,manual,159 gr/m3,Rojo,SÍ,Profesional,7539756,Renault,9450,405,430,147,181,5,190,6.9,9.3,5.5,10.9,0,1598,4 en línea,Manual,151,5,common,sevilla,https://www.coches.com/coches-segunda-mano/oca...
4,10/2020,75 CV,30.086 km,Gasolina,5 puertas,manual,0 gr/m3,Rojo,SÍ,Profesional,7526080,Opel,14999,309,406,144,177,5,174,0.0,0.0,0.0,13.2,0,1199,3 en línea,Manual,118,5,common,sevilla,https://www.coches.com/coches-segunda-mano/oca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16865,12/2020,150 CV,16.665 km,Diesel,4 puertas,manual,144 gr/m3,Rojo,SÍ,Profesional,7305409,Peugeot,36700,0,496,190,193,5,160,5.7,6.4,5.3,13.4,0,1997,4 en línea,Manual,370,6,commercial,sevilla,https://www.coches.com/coches-segunda-mano/oca...
16866,06/2016,100 CV,192.175 km,Diesel,5 puertas,manual,113 gr/m3,Azul,SÍ,Profesional,7290234,Citroen,11200,0,438,180,181,5,166,4.3,5.1,3.9,12.4,0,1560,4 en línea,Manual,254,5,commercial,sevilla,https://www.coches.com/coches-segunda-mano/oca...
16867,10/2018,150 CV,55.016 km,Diesel,4 puertas,manual,144 gr/m3,Gris Plata,SÍ,Profesional,7286674,Peugeot,33580,0,496,190,193,5,160,5.7,6.4,5.3,13.4,0,1997,4 en línea,Manual,370,6,commercial,sevilla,https://www.coches.com/coches-segunda-mano/oca...
16868,05/2018,95 CV,49.078 km,Diesel,4 puertas,manual,104 gr/m3,Blanco,SÍ,Profesional,7279698,Ford,16215,0,416,173,198,5,0,4.1,4.7,3.7,0.0,0,1499,4 en línea,Manual,0,5,commercial,sevilla,https://www.coches.com/coches-segunda-mano/oca...
